In [1]:
import numpy as np 
import pandas as pd
import requests 
from bs4 import BeautifulSoup

In [2]:
base_url = 'https://understat.com/league'
leagues = ['La_Liga', 'Bundesliga', 'EPL', 'Serie_A', 'Ligue_1']
seasons = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']

In [3]:
url = base_url+'/'+leagues[0]+'/'+seasons[9] 
res = requests.get(url)
soup = BeautifulSoup(res.content, "lxml")
scripts = soup.find_all('script')

In [4]:
import json
string_with_json_obj = ''
for el in scripts:
    if 'teamsData' in el.text:
        string_with_json_obj = el.text.strip()

In [5]:
#deleting symbols

ind_start = string_with_json_obj.index("('")+2 
ind_end = string_with_json_obj.index("')") 
json_data = string_with_json_obj[ind_start:ind_end] 
json_data = json_data.encode('utf8').decode('unicode_escape')

#print(string_with_json_obj)

In [6]:
data = json.loads(json_data)

In [7]:
df1 = pd.DataFrame(data.values())

print(df1)

     id            title                                            history
0   138          Sevilla  [{'h_a': 'h', 'xG': 0.714148, 'xGA': 1.06796, ...
1   140    Real Sociedad  [{'h_a': 'h', 'xG': 1.46019, 'xGA': 1.27791, '...
2   142           Getafe  [{'h_a': 'h', 'xG': 0.804505, 'xGA': 1.67817, ...
3   143  Atletico Madrid  [{'h_a': 'h', 'xG': 3.01246, 'xGA': 1.13452, '...
4   144       Las Palmas  [{'h_a': 'h', 'xG': 0.871699, 'xGA': 1.49411, ...
5   145   Rayo Vallecano  [{'h_a': 'a', 'xG': 2.09532, 'xGA': 1.1196, 'n...
6   146         Valencia  [{'h_a': 'a', 'xG': 1.06796, 'xGA': 0.714148, ...
7   147    Athletic Club  [{'h_a': 'h', 'xG': 0.700577, 'xGA': 1.33266, ...
8   148        Barcelona  [{'h_a': 'a', 'xG': 1.67817, 'xGA': 0.804505, ...
9   150      Real Madrid  [{'h_a': 'a', 'xG': 1.33266, 'xGA': 0.700577, ...
10  152       Celta Vigo  [{'h_a': 'h', 'xG': 0.783184, 'xGA': 0.964645,...
11  153       Real Betis  [{'h_a': 'a', 'xG': 0.969349, 'xGA': 0.737812,...
12  154     

In [8]:
teams = {}
for id in data.keys():
    teams[id] = data[id]['title']
    

In [9]:
columns = []
values = []
for id in data.keys():
  columns = list(data[id]['history'][0].keys())
  values = list(data[id]['history'][0].values())
  break

In [10]:
# Getting data for all teams
dataframes = {}
for id, team in teams.items():
  teams_data = []
  for row in data[id]['history']:
    teams_data.append(list(row.values()))
    
  df = pd.DataFrame(teams_data, columns=columns)
  dataframes[team] = df
  print('Added data for {}.'.format(team))

Added data for Sevilla.
Added data for Real Sociedad.
Added data for Getafe.
Added data for Atletico Madrid.
Added data for Las Palmas.
Added data for Rayo Vallecano.
Added data for Valencia.
Added data for Athletic Club.
Added data for Barcelona.
Added data for Real Madrid.
Added data for Celta Vigo.
Added data for Real Betis.
Added data for Villarreal.
Added data for Granada.
Added data for Osasuna.
Added data for Alaves.
Added data for Almeria.
Added data for Girona.
Added data for Mallorca.
Added data for Cadiz.


In [11]:
for team, df in dataframes.items(): 
    dataframes[team]['ppda_coef'] = dataframes[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
    dataframes[team]['oppda_coef'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)

In [12]:
cols_to_sum = ['xG', 'xGA', 'npxG', 'npxGA', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'wins', 'draws', 'loses', 'pts', 'npxGD']
cols_to_mean = ['ppda_coef', 'oppda_coef']

In [13]:
frames = []
for team, df in dataframes.items():
  sum_data = pd.DataFrame(df[cols_to_sum].sum()).transpose()
  mean_data = pd.DataFrame(df[cols_to_mean].mean()).transpose()
  final_df = sum_data.join(mean_data)
  final_df['team'] = team
  final_df['matches'] = len(df)
  frames.append(final_df)
  
full_stat = pd.concat(frames)

In [14]:
full_stat = full_stat[['team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'npxG', 'xGA', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts']]
full_stat.sort_values('pts', ascending=False, inplace=True)
full_stat.reset_index(inplace=True, drop=True)
full_stat['position'] = range(1,len(full_stat)+1)

In [15]:
full_stat['xG_diff'] = full_stat['xG'] - full_stat['scored']
full_stat['xGA_diff'] = full_stat['xGA'] - full_stat['missed']
full_stat['xpts_diff'] = full_stat['xpts'] - full_stat['pts']

In [16]:
cols_to_int = ['wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'deep', 'deep_allowed']
full_stat[cols_to_int] = full_stat[cols_to_int].astype(int)

In [17]:
col_order = ['position','team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'xG_diff', 'npxG', 'xGA', 'xGA_diff', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts', 'xpts_diff']
full_stat = full_stat[col_order]
full_stat.columns = ['#', 'team', 'M', 'W', 'D', 'L', 'G', 'GA', 'PTS', 'xG', 'xG_diff', 'NPxG', 'xGA', 'xGA_diff', 'NPxGA', 'NPxGD', 'PPDA', 'OPPDA', 'DC', 'ODC', 'xPTS', 'xPTS_diff']
pd.options.display.float_format = '{:,.2f}'.format
full_stat.head(10)

,#,team,M,W,D,L,G,GA,PTS,xG,...,xGA,xGA_diff,NPxGA,NPxGD,PPDA,OPPDA,DC,ODC,xPTS,xPTS_diff
0,1,Real Madrid,38,29,8,1,87,26,95,80.88,...,39.29,13.29,38.55,38.55,13.09,17.73,381,167,77.38,-17.62
1,2,Barcelona,38,26,7,5,79,44,85,81.76,...,45.44,1.44,43.95,32.61,8.60,17.76,380,213,76.96,-8.04
2,3,Girona,38,25,6,7,85,46,81,78.05,...,61.93,15.93,56.72,15.22,12.36,15.94,308,233,63.78,-17.22
3,4,Atletico Madrid,38,24,4,10,70,43,76,69.83,...,43.46,0.46,41.97,23.99,13.96,15.99,324,223,69.85,-6.15
4,5,Athletic Club,38,19,11,8,61,37,68,61.64,...,42.03,5.03,38.31,19.61,10.97,11.89,244,166,62.92,-5.08
5,6,Real Sociedad,38,16,12,10,51,39,60,49.16,...,47.29,8.29,42.09,4.10,7.76,13.65,201,173,52.70,-7.30
6,7,Real Betis,38,14,15,9,48,45,57,52.92,...,52.48,7.48,46.38,4.31,13.40,10.89,286,205,54.65,-2.35
7,8,Villarreal,38,14,11,13,65,65,53,59.81,...,65.86,0.86,61.41,-6.80,13.44,11.11,269,295,50.54,-2.46
8,9,Valencia,38,13,10,15,40,45,49,45.13,...,52.33,7.33,47.87,-9.43,15.94,10.25,138,241,47.96,-1.04
9,10,Alaves,38,12,10,16,36,46,46,51.97,...,52.08,6.08,46.14,0.63,13.91,8.57,179,233,50.72,4.72


In [18]:
season_data = dict()
season_data[seasons[4]] = full_stat
print(season_data)
full_data = dict()
full_data[leagues[0]] = season_data
print(full_data)

{'2018':      #             team   M   W   D   L   G  GA  PTS    xG  ...   xGA  \
0    1      Real Madrid  38  29   8   1  87  26   95 80.88  ... 39.29   
1    2        Barcelona  38  26   7   5  79  44   85 81.76  ... 45.44   
2    3           Girona  38  25   6   7  85  46   81 78.05  ... 61.93   
3    4  Atletico Madrid  38  24   4  10  70  43   76 69.83  ... 43.46   
4    5    Athletic Club  38  19  11   8  61  37   68 61.64  ... 42.03   
5    6    Real Sociedad  38  16  12  10  51  39   60 49.16  ... 47.29   
6    7       Real Betis  38  14  15   9  48  45   57 52.92  ... 52.48   
7    8       Villarreal  38  14  11  13  65  65   53 59.81  ... 65.86   
8    9         Valencia  38  13  10  15  40  45   49 45.13  ... 52.33   
9   10           Alaves  38  12  10  16  36  46   46 51.97  ... 52.08   
10  11          Osasuna  38  12   9  17  45  56   45 45.79  ... 57.37   
11  12           Getafe  38  10  13  15  42  54   43 50.28  ... 57.65   
12  13          Sevilla  38  10  11  17  4

In [19]:
full_data = dict()
for league in leagues:
  
  season_data = dict()
  for season in seasons:    
    url = base_url+'/'+league+'/'+season
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "lxml")

    scripts = soup.find_all('script')
    
    string_with_json_obj = ''

    #Find data for teams
    for el in scripts:
        if 'teamsData' in el.text:
          string_with_json_obj = el.text.strip()


    #strip unnecessary symbols and get only JSON data
    ind_start = string_with_json_obj.index("('")+2
    ind_end = string_with_json_obj.index("')")
    json_data = string_with_json_obj[ind_start:ind_end]
    json_data = json_data.encode('utf8').decode('unicode_escape')
    
    
    #convert JSON data into Python dictionary
    data = json.loads(json_data)
    
    #Get teams and their relevant ids and put them into separate dictionary
    teams = {}
    for id in data.keys():
      teams[id] = data[id]['title']
      
    # EDA to get a feeling of how the JSON is structured
    # Column names are all the same, so we just use first element
    columns = []
    # Check the sample of values per each column
    values = []
    for id in data.keys():
      columns = list(data[id]['history'][0].keys())
      values = list(data[id]['history'][0].values())
      break
      
    # Getting data for all teams
    dataframes = {}
    for id, team in teams.items():
      teams_data = []
      for row in data[id]['history']:
        teams_data.append(list(row.values()))

      df = pd.DataFrame(teams_data, columns=columns)
      dataframes[team] = df
      
    
    for team, df in dataframes.items():
      dataframes[team]['ppda_coef'] = dataframes[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
      dataframes[team]['oppda_coef'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
      
    cols_to_sum = ['xG', 'xGA', 'npxG', 'npxGA', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'wins', 'draws', 'loses', 'pts', 'npxGD']
    cols_to_mean = ['ppda_coef', 'oppda_coef']
    
    frames = []
    for team, df in dataframes.items():
      sum_data = pd.DataFrame(df[cols_to_sum].sum()).transpose()
      mean_data = pd.DataFrame(df[cols_to_mean].mean()).transpose()
      final_df = sum_data.join(mean_data)
      final_df['team'] = team
      final_df['matches'] = len(df)
      frames.append(final_df)

    full_stat = pd.concat(frames)
    
    full_stat = full_stat[['team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'npxG', 'xGA', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts']]
    full_stat.sort_values('pts', ascending=False, inplace=True)
    full_stat.reset_index(inplace=True, drop=True)
    full_stat['position'] = range(1,len(full_stat)+1)
    
    full_stat['xG_diff'] = full_stat['xG'] - full_stat['scored']
    full_stat['xGA_diff'] = full_stat['xGA'] - full_stat['missed']
    full_stat['xpts_diff'] = full_stat['xpts'] - full_stat['pts']
    
    cols_to_int = ['wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'deep', 'deep_allowed']
    full_stat[cols_to_int] = full_stat[cols_to_int].astype(int)
    
    col_order = ['position', 'team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'xG_diff', 'npxG', 'xGA', 'xGA_diff', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts', 'xpts_diff']
    full_stat = full_stat[col_order]
    full_stat = full_stat.set_index('position')
    # print(full_stat.head(20))
    
    season_data[season] = full_stat
  
  df_season = pd.concat(season_data)
  full_data[league] = df_season
  
data = pd.concat(full_data)
data.head()
  

team  matches  wins  draws  loses  scored  \
             position                                                         
La_Liga 2014 1               Barcelona       38    30      4      4     110   
             2             Real Madrid       38    30      2      6     118   
             3         Atletico Madrid       38    23      9      6      67   
             4                Valencia       38    22     11      5      70   
             5                 Sevilla       38    23      7      8      71   

                       missed  pts     xG  xG_diff  ...   xGA  xGA_diff  \
             position                               ...                   
La_Liga 2014 1             21   94 102.98    -7.02  ... 28.44      7.44   
             2             38   92  95.77   -22.23  ... 42.61      4.61   
             3             29   78  57.05    -9.95  ... 29.07      0.07   
             4             32   77  55.06   -14.94  ... 39.39      7.39   
             5             45   76  69.53    -1.47  ... 47.86      2.86   

                       npxGA  npxGD  ppda_coef  oppda_coef  deep  \
             position                                              
La_Liga 2014 1         24.73  73.05       5.68       16.37   489   
             2         38.89  47.21      10.21       12.93   351   
             3         26.84  25.75       8.98        9.24   197   
             4         33.45  16.26       8.71        7.87   203   
             5         41.92  20.18       8.28        9.48   305   

                       deep_allowed  xpts  xpts_diff  
             position                                 
La_Liga 2014 1                  114 94.08       0.08  
             2                  153 81.75     -10.25  
             3                  123 73.14      -4.86  
             4                  172 63.71     -13.29  
             5                  168 67.39      -8.61  

[5 rows x 21 columns]

In [20]:


data.to_csv('understat1.com.csv')

